#    APRICOT: Advanced Platform for Reproducible Infrastructures in the Cloud via Open Tools

This notebook provide a complete example of reproducible experimentation in the field of life science. The objective is to determine the best reconstruction parameters for a simulated Positron Emision Tomography (PET) scaner system.

# Experimentation

TO DO

## Setting up the system

For this experimentation we need perform a multiparametric analysis to obtain the best image reconstruction parameters. So, the first step is infrastructure deployment. The chosen infrastructure is a batch cluster with 5 worker nodes and ubuntu 18.04 LTS images as OS. Used nodes have 2 GB of RAM and 20 of disk space. Also, we recommend to assign, at least, two CPUs to front-end node for this experiment. To reproduce the experiment deploy this infrastructure using the APRICOT deploy extension and one of the supported cloud providers. Once the cluster has been configured, you can follow the experimentation.

Store the cluster and user names

In [ ]:
clusterName = "reconstruction"

In [1]:
username = "ubuntu"

We need to load the python module with the APRICOT magics to manage our clusters. This step can be avoided if the module is loaded by default on each notebook.

In [ ]:
%reload_ext apricot_magic

Once the infrastructure has been deployed and configured, check it using %apricot_ls magic

In [ ]:
%apricot_ls

If the infrastructure deployment fails, we can get the output log with the following instruction,

In [ ]:
%apricot_log $clusterName

## Preparing data and programs

Now, we must provide the necessary data and programs for our analisys:

1- Raw simulated data

2- Ideal image

3- Comparision program code

4- Reconstruction program code


All required files can be upload easily from our computer using the ''%apricot_upload'' instruction. First, check if the data is at the current directory:

In [2]:
%%bash
ls

input
reconstruction.ipynb
script.sh


Next, upload files. This can take several minutes.

In [ ]:
%apricot_upload $clusterName input /home/$username

Check if all the files has been succesfully uploaded

In [ ]:
%apricot exec $clusterName ls /home/$username/input

Now, we need to compile the source codes. All the necessary compilers and cmake tools should be installed at configuration initialization.

Compile the reconstruction and comparision programs

In [ ]:
%apricot exec $clusterName cd input/reconstructor_code && bash install.sh && cp reconstructor ../

In [ ]:
%apricot exec $clusterName cd input && g++ -o evaluateImage evaluateImage.cpp -O2

Check if the executables has been created (evaluateImage and reconstructor)

In [ ]:
%apricot exec $clusterName ls input/

Now we have all necessary files at cluster. This has been configured automatically with nfs, so the '/home' directory is shared by all workers and the frontend.

We need also a folder to store the results. Create a folder with 'results' name.

In [ ]:
%apricot exec $clusterName mkdir results

## Executing jobs

Now, is time to execute the experiment. For simplicity, this multiparametric study only uses three variable parameters. However, this can be extended to any number of parameter ranges. First of all, we need to specify each parameter interval and step size.

In [ ]:
minNvox_xy = 20
maxNvox_xy = 250
stepNvox_xy = 20

minNvox_z = 20
maxNvox_z = 250
stepNvox_z = 20

nChunksMin = 3
nChunksMax = 5
chunkStep = 1

Now, get a identifier for this ranges

In [ ]:
ID = %apricot_genMPid $minNvox_xy $maxNvox_xy $stepNvox_xy $minNvox_z $maxNvox_z $stepNvox_z $nChunksMin $nChunksMax $chunkStep

In [ ]:
print(ID)

Create a specific folder for results with previous ID

In [ ]:
%apricot exec $clusterName mkdir results/$ID

In [ ]:
%apricot exec $clusterName ls results

Launch the jobs using ''%apricot_runMP'' and the script ''script.sh''. This step can be delayed several minutes until all workers has been configured.

In [ ]:
%apricot_runMP $clusterName script.sh $minNvox_xy $maxNvox_xy $stepNvox_xy $minNvox_z $maxNvox_z $stepNvox_z $nChunksMin $nChunksMax $chunkStep
            

Check if all jobs has been finished

In [ ]:
%apricot exec $clusterName squeue

When no jobs appear in the tasks queue, execute post-processing script

In [ ]:
%apricot exec $clusterName bash input/getResults.sh $ID

## Getting results

At next step, download the file with the results of our multiparametric study.

In [ ]:
resultsFilename = "results-" + ID + ".dat"
%apricot_download $clusterName $resultsFilename .

Read results file data

In [ ]:
fileIn = open(resultsFilename,"r")

data = fileIn.read()

# Extract data lines
data = data.strip().split('\n')
# Remove header line
data.pop(0)

# Extract input data
XYnvox = []
Znvox = []
chunks = []
userTimeMin = []
userTimeSec = []
systemTimeMin = []
systemTimeSec = []
RMSE = []
PSNR = []
NRMSD = []
NMAD = []

for line in data:
    line = " ".join(line.split())
    words = line.strip().split(' ')
    XYnvox.append(float(words[0]))
    Znvox.append(float(words[1]))
    chunks.append(float(words[2]))
    userTimeMin.append(float(words[3]))
    userTimeSec.append(float(words[4]))
    systemTimeMin.append(float(words[5]))
    systemTimeSec.append(float(words[6]))
    RMSE.append(float(words[7]))
    PSNR.append(float(words[8]))
    NRMSD.append(float(words[9]))
    NMAD.append(float(words[10]))
    
    
fileIn.close()

Import numpy and plot libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

userTime = np.add(np.multiply(userTimeMin,60.0),userTimeSec)
systTime = np.add(np.multiply(systemTimeMin,60.0),systemTimeSec)
totalTime = userTime + systTime

Plot the results

In [ ]:

nChunks = 5.0
subXY = []
subZ = []
subTimes = []
subRMSE = []
subPSNR = []
subNRMSD = []
subNMAD = []

subRMSE_zoom = []
subXY_zoom = []
subZ_zoom = []
nVoxCut = 60

for i in list(range(len(XYnvox))):
    if nChunks == chunks[i]:
        subXY.append(XYnvox[i])
        subZ.append(Znvox[i])
        subTimes.append(totalTime[i])
        subRMSE.append(RMSE[i])
        subPSNR.append(PSNR[i])
        subNRMSD.append(NRMSD[i])
        subNMAD.append(NMAD[i])
        if XYnvox[i] > nVoxCut and Znvox[i] > nVoxCut:
            subXY_zoom.append(XYnvox[i])
            subZ_zoom.append(Znvox[i])
            subRMSE_zoom.append(RMSE[i])
        
Axpad = 280        
fig = plt.figure()
plt.rcParams["figure.figsize"] = [100,100]
plt.rcParams.update({'font.size': 128})

ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('nº voxels x-y')
ax.set_ylabel('nº voxels z')
ax.set_zlabel('time (s)')

ax.xaxis.labelpad = Axpad
ax.yaxis.labelpad = Axpad
ax.zaxis.labelpad = Axpad


ax.plot_trisurf(subXY,subZ,subTimes)
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('nº voxels x-y')
ax.set_ylabel('nº voxels z')
ax.set_zlabel('rmse')

ax.xaxis.labelpad = Axpad
ax.yaxis.labelpad = Axpad
ax.zaxis.labelpad = Axpad

ax.plot_trisurf(subXY,subZ,subRMSE)
plt.show()


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('nº voxels x-y')
ax.set_ylabel('nº voxels z')
ax.set_zlabel('rmse')

ax.xaxis.labelpad = Axpad
ax.yaxis.labelpad = Axpad
ax.zaxis.labelpad = Axpad

ax.plot_trisurf(subXY_zoom,subZ_zoom,subRMSE_zoom)
plt.show()


## Delete infrastructure

If we have been finished the experimentation, destroy the cluster

In [ ]:
%apricot destroy $clusterName

# Conclusions and future work

TODO

# References

TODO